In [ ]:
# NO Modificar
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# NO Modificar
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
# NO Modificar
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

## Definimos la red neuronal
Podemos utilizar las siguientes capas
- layers.Conv2D
    - Seleccionar activación, tamaño de kernel y número de canales
- layers.MaxPooling2D
    - Seleccionar tamaño de kernel
- layers.Flatten
- layers.Dense
- layers.Softmax

## Extra
Comparar resultados de una red neuronal totalmente conectada con una convolucional


In [ ]:
model = models.Sequential()
# Convolutional part
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Otra arquitectura que puede mejorar la parte convolucional
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
# model.add(layers.Dropout(0.25))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
# model.add(layers.Dropout(0.25))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
# model.add(layers.Dropout(0.25))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))

# Fully connected part
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))
model.add(layers.Softmax())

In [ ]:
# Visualizamos resumen de la red
model.summary()

## Entrenamiento
Escoger los siguientes valores:
- batch_size
- epochs
- validation_split

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=10, verbose = 1, mode = 'min', restore_best_weights = True)

# Llenar estos valores
batch_size = 64
epochs = 20
validation_split = 0.2

history = model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs, validation_split=validation_split, callbacks=[early_stopping])

In [ ]:
# # NO Modificar: Solamente genera gráficas
plt.figure()
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(which="major", linestyle='-')
plt.minorticks_on()
plt.grid(which="minor", linestyle='-.')

plt.figure()
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(which="major", linestyle='-')
plt.minorticks_on()
plt.grid(which="minor", linestyle='-.')



In [ ]:
# Computamos exactitud y costo del modelo entrenado 
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(f"Test Accuracy: {test_acc}")
print(f"Test Loss: {test_loss}")

In [ ]:
# NO Modificar
# Predecimos las clases del conjunto de prueba
# Esto va a generar predicciones para todas las imágenes de prueba
predictions = model.predict(test_images)

# Visualicemos una predicción
print(f"Número total de predicciones: {predictions.shape[0]}")
print("Primera predicción (probabilidades)")
print(predictions[0])

In [ ]:
# NO modificar Funciones auxiliares para generar gráficas
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label.flatten()[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  # IMPORTANTE: Aquí seleccionamos la clase con mayor probabilidad
  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label.flatten()[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  # IMPORTANTE: Aquí seleccionamos la clase con mayor probabilidad
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# Generamos visualización de las predicciones de las primeras imágenes del conjunto de prueba
num_rows = 5
num_cols = 4
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()
